In [1]:
!pip install python-dotenv
# pip install python-dotenv


  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


In [10]:
import os
from dotenv import load_dotenv

load_dotenv()
pinecone_api_key = os.getenv("PINECONE_API_KEY")

if pinecone_api_key is None:
    raise ValueError("PINECONE_API_KEY not found in the environment variables.Please check your .env file.")
else:
    print(pinecone_api_key)

d57c27b0-496e-48cb-b15b-d94359627f11


In [12]:
import pinecone
pinecone.init(api_key=os.getenv("PINECONE_API_KEY"),
              environment=os.getenv('PINECONE_API_ENV'))


In [14]:
index_name = "chatbot"
pinecone.create_index(index_name, dimension=384)

### Loading the data

In [15]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

def load_pdf(data):
    try:
        loader = DirectoryLoader(data,
                        glob='*.pdf',
                        loader_cls=PyPDFLoader)
        
        documents = loader.load()
        return documents
    
    except Exception as e:
        raise e
    

In [16]:
path = r"F:\End_To_End_project\Genrative_Ai_Project\Madical _Chat_Bot\data"
loaded_data = load_pdf(path)

### Converting the data into 500-500 Chunks

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def text_spliter_into_chunks(loaded_data):
    try:
        text_spliter = RecursiveCharacterTextSplitter(chunk_size = 500,
                                    chunk_overlap=20
                                    )
        
        text_chunks = text_spliter.split_documents(loaded_data)

        return text_chunks
    except Exception as e:
        raise e

In [18]:
chunks = text_spliter_into_chunks(loaded_data=loaded_data)
print(f"number of chunks from, {len(chunks)}")

number of chunks from, 7020


## I am dowloading the Embddings

In [19]:
from langchain.embeddings import HuggingFaceEmbeddings
def downlaod_hugging_face_embedding():
    embedding = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    return embedding

In [20]:
embadding = downlaod_hugging_face_embedding()
embadding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [22]:
from langchain.vectorstores import Pinecone
index_name="chatbot"
docsearch = Pinecone.from_texts([chunk.page_content for chunk in chunks],
                                embedding=embadding,index_name=index_name)
